In [18]:
!pip install google-genai pydantic pandas

from google import genai
from pydantic import BaseModel, Field
from typing import List, Literal
import pandas as pd
import os

# 1. Initialize the NEW Client
# Ensure GOOGLE_API_KEY is set in your environment or Colab secrets
client = genai.Client(api_key=os.environ.get(GOOGLE_API_KEY))

# 2. Define Output Schema
class SentimentOutput(BaseModel):
    review_id: int
    sentiment: Literal['positive', 'negative']
    # FIX 1: Removed le=1.0/ge=0.0 to prevent "Schema has no 'maximum' field" error
    confidence: float = Field(description="Model confidence score between 0.0 and 1.0")
    # FIX 2: Changed min_items to min_length for Pydantic V2 compatibility
    emotions: List[Literal['joy', 'sadness', 'disgust', 'anger', 'fear']] = Field(min_length=1)

# 3. Sample Data
raw_reviews = [
    {"id": 1, "text": "Finally found a routine that works! My acne is gone and I feel confident."},
    {"id": 2, "text": "The exam room floor was sticky and I saw used gauze. It made me sick."},
    {"id": 3, "text": "Spent thousands on laser treatments with zero improvement. Hopeless."},
    {"id": 4, "text": "Dr. Lee is nice, but the waiting room smells like old eggs."},
    {"id": 5, "text": "The procedure was okay. The nurse was a bit rude, but results are fine."}
]

# 4. Analysis Function
def analyze_review(review_id: int, text: str) -> SentimentOutput:
    try:
        response = client.models.generate_content(
            model='gemini-2.0-flash',
            contents=f"Analyze this review: '{text}'",
            config={
                'response_mime_type': 'application/json',
                'response_schema': SentimentOutput,
            }
        )
        # The new SDK automatically parses the JSON into your Pydantic model
        # Access it via .parsed
        result = response.parsed

        # Inject ID back (optional, depending on if you want the model to predict it)
        if result:
            result.review_id = review_id
            return result

    except Exception as e:
        print(f"Error processing ID {review_id}: {e}")
        return None

# 5. Process and Display
results = []
print("Analyzing with google-genai SDK...")

for item in raw_reviews:
    analysis = analyze_review(item['id'], item['text'])
    if analysis:
        results.append(analysis.model_dump())

# 6. Create Data Frame
df = pd.DataFrame(results)
print(df[['sentiment', 'confidence', 'emotions']])

NameError: name 'GOOGLE_API_KEY' is not defined